In [56]:
import requests

In [57]:
# for 'Barton Springs & S Lamar Blvd, Austin'
# http://www.mapquestapi.com/geocoding/v1/address?key=AG6E2IsTpxqw4QO2PEoxCl4b1Q4Iwl7Q&location=Barton+Springs+Rd+&+S+Lamar+Blvd+,+Austin

In [58]:
key = 'AG6E2IsTpxqw4QO2PEoxCl4b1Q4Iwl7Q'
location = 'Barton Springs Rd & S Lamar Blvd, Austin, TX'

In [59]:
base_url = f'http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={location}'

In [60]:
res = requests.get(base_url)

In [ ]:
# In the above cell I previously had the following. Consider putting the params dictionary back in and creating
# a list or series of locations that can be entered as [locations] or whatever in place of the value 'Austin,TX'

# res = requests.get(base_url,
#                   params={'location': 'Austin,TX',
#                           'apikey': key})

In [61]:
res.status_code

200

In [62]:
res.json()

# the one we need for this example is the 4th one...hmm...

{'info': {'statuscode': 0,
  'copyright': {'text': '© 2020 MapQuest, Inc.',
   'imageUrl': 'http://api.mqcdn.com/res/mqlogo.gif',
   'imageAltText': '© 2020 MapQuest, Inc.'},
  'messages': []},
 'options': {'maxResults': -1, 'thumbMaps': True, 'ignoreLatLngInput': False},
 'results': [{'providedLocation': {'location': 'Barton Springs Rd '},
   'locations': [{'street': '[1 - 599] Barton Springs Rd',
     'adminArea6': '',
     'adminArea6Type': 'Neighborhood',
     'adminArea5': 'Normandy',
     'adminArea5Type': 'City',
     'adminArea4': 'Bedford',
     'adminArea4Type': 'County',
     'adminArea3': 'TN',
     'adminArea3Type': 'State',
     'adminArea1': 'US',
     'adminArea1Type': 'Country',
     'postalCode': '37360',
     'geocodeQualityCode': 'B1AAA',
     'geocodeQuality': 'STREET',
     'dragPoint': False,
     'sideOfStreet': 'M',
     'linkId': 'rnr843340',
     'unknownInput': '',
     'type': 's',
     'latLng': {'lat': 35.454548, 'lng': -86.220656},
     'displayLatLng': 